In [1]:
#Experimento con más parámetros
import requests
import time
import pandas as pd
import time
count = 0
contador=0

In [57]:
def connect_to_endpoint(bearer_token, query, next_token=None):
    
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    
    # add additional parameters as needed
    params = {"tweet.fields" : "author_id,created_at,entities,public_metrics,referenced_tweets,source","expansions":"author_id,entities.mentions.username,referenced_tweets.id","user.fields":"created_at,description,entities,id,location,name,protected,public_metrics,url,username,verified"}

    # replace appropriate start and end times below 
    #I have to modify the timeperiod start_time and end_time -----------------------------------
    if (next_token is not None):
        url = "https://api.twitter.com/2/tweets/search/all?max_results=500&query={}&start_time=2017-01-01T00:00:00Z&end_time=2021-12-31T23:59:59Z&next_token={}".format(query, next_token)
    else:
        url = "https://api.twitter.com/2/tweets/search/all?max_results=500&start_time=2017-01-01T00:00:00Z&end_time=2021-12-31T23:59:59Z&query={}".format(query)
    response = requests.request("GET", url, params=params , headers=headers)
    
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    
    return response.json()


In [58]:
def twitter_call(cuenta):
    
    global count
    global contador
    flag = True

    # Replace with your own bearer token from your academic project in developer portal
    #I ahve to replace my credentials API -----------------------------------------------------------
    bearer_token = "AAAAAAAAAAAAAAAAAAAAAKdwjAEAAAAAOwIYVizquWN0gcU5PDgXypDRlx0%3DxFZ8Gys1Zk0f9Ehv8FasTUDw3Epa24Ke0b9aCtYciP30HkIJcy"


    json_response = connect_to_endpoint(bearer_token, cuenta, None)


    while flag:
        # Replace the count below with the number of Tweets you want to stop at. 
        # Note: running without the count check will result in getting more Tweets
        # that will count towards the Tweet cap
        if contador >= 280:
            time.sleep(1000)
        result_count = json_response['meta']['result_count']

        if 'next_token' in json_response['meta']:
            next_token = json_response['meta']['next_token']
            if result_count is not None and result_count > 0 and next_token is not None:

                df = (pd.DataFrame(json_response['data'])).sort_index(axis = 1)
                df.to_csv('MaltaTweets.csv', mode="a", header=False)
                
                df2 = (pd.DataFrame(json_response['includes']['users'])).sort_index(axis = 1)
                df2.to_csv('MaltaaUsers.csv', mode="a", header=False)

                
                
                count += result_count
                contador += 1
                print(contador)
                json_response = connect_to_endpoint(bearer_token, cuenta, next_token)

        else:
            flag = False

    df = (pd.DataFrame(json_response['data'])).sort_index(axis = 1)
    df.to_csv('MaltaTweets.csv', mode="a", header=False)
    
    df2 = (pd.DataFrame(json_response['includes']['users'])).sort_index(axis = 1)
    df2.to_csv('MaltaUsers.csv', mode="a", header=False)

    
    
    result_count = json_response['meta']['result_count']
    count += result_count
    contador += 1
    print("Total Tweet IDs saved: {}".format(contador))
 

# Specify keywords

In [59]:
a="open.data.gov.mt"
b="datagouvfr"

In [60]:
#cuenta= "(url:\""+a+"\" OR @"+b+") -is:retweet -is:reply -is:quote"  #all the tweets mentioning the account
cuenta= "(url:\""+a+"\") -is:retweet -is:reply -is:quote"  #all the tweets mentioning the account
#cuenta= "(from:\""+b+"\")" #all the tweets by an account 
twitter_call(cuenta)

KeyError: 'data'

In [50]:
df=pd.read_csv("MaltaTweets2.csv", header=None, engine='python',error_bad_lines=False)
df.columns = ['0','author_id','created_time','3','entities','tweet_id','metrics','source','text']
#df=df.drop_duplicates(subset=['tweet_id'])
#df= pd.concat([df,df['metrics'].map(eval).apply(pd.Series)],axis=1)
df

C:\Users\Abdul Aziz\.conda\envs\pycarete\lib\site-packages\IPython\core\interactiveshell.py:3457: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


,0,author_id,created_time,3,entities,tweet_id,metrics,source,text
0,0,3593185034,2020-03-26T14:47:27.000Z,['1243187815451521024'],"{'urls': [{'start': 51, 'end': 74, 'url': 'htt...",1243187815451521024,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",TweetDeck,【世界のジオポータル】マルタ\nMalta Information Technology A...


In [ ]:
#df.to_excel("tweets.xlsx")

In [51]:
len(df["author_id"].unique())

1

In [52]:
df2=pd.read_csv("MaltaUsers2.csv", header=None, engine='python',error_bad_lines=False)
#df3=pd.read_csv("users1.csv", header=None, engine='python',error_bad_lines=False, skiprows = 7735, nrows=610)
#df2=df2.append(df3)
df2

C:\Users\Abdul Aziz\.conda\envs\pycarete\lib\site-packages\IPython\core\interactiveshell.py:3457: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


,0,1,2,3,4,5,6,7,8,9,10,11
0,0,2015-09-17T11:04:23.000Z,いい年した大人（「社会科」教員免許失効済み非教育業界、残念な地理好き）のお勉強垢◆天気予報図...,"{'url': {'urls': [{'start': 0, 'end': 23, 'url...",3593185034,お品書き↓,地理総合さん＠地図萌え,False,"{'followers_count': 295, 'following_count': 22...",https://t.co/PO4XsV1RcS,Ms_Kisyo_GEO,False


In [ ]:
#df2.to_excel("usuarios.xlsx")

In [53]:
df2=df2[[4,10,11]]
df2.columns = ["author_id","username","verified"]
df2=df2.drop_duplicates(subset=['author_id'])
df2

,author_id,username,verified
0,3593185034,Ms_Kisyo_GEO,False


In [54]:
#df['author_id'] = df['author_id'].astype(float)
df['author_id'] =df['author_id'] .apply(lambda col:pd.to_numeric(col, errors='coerce'))
df2['author_id'] =df2['author_id'] .apply(lambda col:pd.to_numeric(col, errors='coerce'))
left_merged = pd.merge(df, df2, how="left", on=["author_id"])
#left_merged=left_merged.drop_duplicates(subset=['author_id'])


verified=len(left_merged[left_merged["verified"]=="True"])


In [55]:
left_merged

,0,author_id,created_time,3,entities,tweet_id,metrics,source,text,username,verified
0,0,3593185034,2020-03-26T14:47:27.000Z,['1243187815451521024'],"{'urls': [{'start': 51, 'end': 74, 'url': 'htt...",1243187815451521024,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",TweetDeck,【世界のジオポータル】マルタ\nMalta Information Technology A...,Ms_Kisyo_GEO,False


In [ ]:
#left_merged.to_excel("tweets.xlsx")

In [56]:
len(left_merged.dropna(subset=['username']))

1